L'objectif de ce notebook est de donner les clés pour utiliser correctement SubQuery

Application sur l'objet ZoneUrba pour calculer la surface artificielle en provenance de l'OCSGE : failed

Application sur l'objet ZoneUrba pour associer le code insee : in progress

[Doc officielle](https://docs.djangoproject.com/en/4.2/ref/models/expressions/#subquery-expressions)

Exemple :
```python
from django.db.models import OuterRef, Subquery
newest = Comment.objects.filter(post=OuterRef("pk")).order_by("-created_at")
Post.objects.annotate(newest_commenter_email=Subquery(newest.values("email")[:1]))
```

In [2]:
from django.db import connection
from django.db.models import OuterRef, Subquery, Sum, F
from django.contrib.gis.db.models.functions import Area, Intersection, Transform
from django.contrib.gis.db.models import Union

from public_data.models import ZoneUrba, Ocsge, Departement, Commune


def execute(query, params=None):
    with connection.cursor() as cursor:
        cursor.execute(query, params)
        return cursor.fetchall()


ZoneUrba.objects.count(), Ocsge.objects.filter(is_artificial=True).count()

(1010442, 464569)

Test sur une seule ZoneUrba

In [3]:
dept = Departement.objects.get(source_id=32)
zone_urba = ZoneUrba.objects.filter(mpoly__intersects=dept.mpoly).first()
zone_urba

<ZoneUrba: ZoneUrba object (266816)>

In [4]:
surface = (
    Ocsge.objects
    .filter(mpoly__intersects=zone_urba.mpoly)
    .annotate(
        intersection=Intersection("mpoly", zone_urba.mpoly),
        intersection_area=Area(Transform("intersection", 2154)) / 10000,
    )
    .values("year")
    .annotate(artificial_area=Sum("intersection_area"))
    .annotate(artificial_area2=Sum(Area(Transform("mpoly", 2154)) / 10000))
    .order_by("-year")
)

surface

<QuerySet [{'year': 2019, 'artificial_area': 0.3919341124533532, 'artificial_area2': 236.22150755014331}, {'year': 2016, 'artificial_area': 0.391933899382672, 'artificial_area2': 236.22154447013892}]>

In [5]:
sub_query = (
    Ocsge.objects
    .annotate(geom=OuterRef("mpoly"))
    .filter(mpoly__intersects=F("geom"))
    .annotate(
        intersection=Intersection("mpoly", F("geom")),
        intersection_area=Area(Transform("intersection", 2154)) / 10000,
    )
    .values("year")
    .annotate(artificial_area=Sum("intersection_area"))
    .order_by("-year")
    .values("artificial_area")
)[:1]

AttributeError: 'ResolvedOuterRef' object has no attribute '_output_field_or_none'

In [ ]:
sub_query = """
SELECT pdo.year, ST_Area(ST_Transform(ST_Union(pdo.mpoly), 2154)) / 10000 as artificial_area
FROM
    public_data_ocsge pdo
    INNER JOIN public_data_zoneurba pdz ON ST_Intersects(pdo.mpoly, pdz.mpoly) AND pdz.id = 266816
GROUP BY pdo.year
ORDER BY pdo.year DESC
LIMIT 1
"""
with connection.cursor() as cursor:
    cursor.execute(sub_query)
    rows = cursor.fetchall()
rows

In [ ]:
query = f"""
UPDATE public_data_zoneurba
SET artificial_area = subquery.artificial_area
FROM (
    SELECT pdz.id as zone_urba_id, ST_Area(ST_Transform(ST_Union(pdo.mpoly), 2154)) / 10000 as artificial_area
    FROM
        public_data_ocsge pdo INNER JOIN public_data_zoneurba pdz ON ST_Intersects(pdo.mpoly, pdz.mpoly)
    GROUP BY pdz.id
    ORDER BY pdz.id DESC
    LIMIT 10
) AS subquery
WHERE subquery.zone_urba_id = id;
"""
with connection.cursor() as cursor:
    cursor.execute(query)

In [ ]:
with connection.cursor() as cursor:
    cursor.execute("SELECT COUNT(*) FROM public_data_zoneurba WHERE artificial_area IS NOT NULL")
    rows = cursor.fetchall()
rows

In [ ]:
query = f"""
UPDATE public_data_zoneurba pdz
SET artificial_area = (
    SELECT ST_Area(ST_Transform(ST_Union(pdo.mpoly), 2154)) / 10000 as artificial_area
    FROM
        public_data_ocsge pdo
    WHERE ST_Intersects(pdo.mpoly, pdz.mpoly)
)
WHERE ;
"""
with connection.cursor() as cursor:
    cursor.execute(query)

In [6]:
q = "SELECT COUNT(*) FROM public_data_zoneurba pdz WHERE ST_IsValid(mpoly) IS FALSE AND ST_IsValid(ST_MakeValid(mpoly))"
execute(q)

[(0,)]

In [7]:
query = """
UPDATE public_data_zoneurba pdz
SET mpoly = ST_Multi(ST_CollectionExtract(ST_MakeValid(mpoly), 3))
WHERE
    ST_IsValid(mpoly) IS FALSE
    AND ST_IsValid(ST_MakeValid(mpoly))
"""
execute(query)

ProgrammingError: no results to fetch

In [ ]:
q = "SELECT COUNT(*) FROM public_data_zoneurba pdz WHERE ST_IsValid(mpoly) IS FALSE"
execute(q)

In [ ]:
from django.db.models import FilteredRelation, Q

ZoneUrba.objects.annotate(ocsge=FilteredRelation(
    "mpoly"
    condition=Q(mpoly__intersects=)
))

Restaurant.objects.annotate(
...     pizzas_vegetarian=FilteredRelation(
...         "pizzas",
...         condition=Q(pizzas__vegetarian=True),
...     ),
... ).filter(pizzas_vegetarian__name__icontains="mozzarella")

In [14]:
sub_query = (
    Ocsge.objects.filter(mpoly__intersects=OuterRef("mpoly"))
    .filter(year="2019")
    .annotate(artificial_area=Sum(Area(Transform("mpoly", 2154))))
)

ZoneUrba.objects.annotate(artif_area=Subquery(sub_query.values("artificial_area")[:1]))

<QuerySet [<ZoneUrba: ZoneUrba object (267015)>, <ZoneUrba: ZoneUrba object (267016)>, <ZoneUrba: ZoneUrba object (267017)>, <ZoneUrba: ZoneUrba object (267018)>, <ZoneUrba: ZoneUrba object (267019)>, <ZoneUrba: ZoneUrba object (267020)>, <ZoneUrba: ZoneUrba object (267108)>, <ZoneUrba: ZoneUrba object (267109)>, <ZoneUrba: ZoneUrba object (267110)>, <ZoneUrba: ZoneUrba object (267111)>, <ZoneUrba: ZoneUrba object (267112)>, <ZoneUrba: ZoneUrba object (267113)>, <ZoneUrba: ZoneUrba object (267114)>, <ZoneUrba: ZoneUrba object (267115)>, <ZoneUrba: ZoneUrba object (267116)>, <ZoneUrba: ZoneUrba object (267021)>, <ZoneUrba: ZoneUrba object (267022)>, <ZoneUrba: ZoneUrba object (267023)>, <ZoneUrba: ZoneUrba object (267024)>, <ZoneUrba: ZoneUrba object (267025)>, '...(remaining elements truncated)...']>

In [17]:
sub_query = (
    Ocsge.objects.filter(year="2019")
    .annotate(geom=OuterRef("mpoly"))
    .filter(mpoly__intersects=F("geom"))
    .annotate(intersection=Intersection("mpoly", F("geom")))
    .annotate(artificial_area=Sum(Area(Transform("mpoly", 2154))))
)

ZoneUrba.objects.annotate(artif_area=Subquery(sub_query.values("artificial_area")[:1]))

AttributeError: 'ResolvedOuterRef' object has no attribute '_output_field_or_none'

In [18]:
query = """
SELECT
    pdz.typezone,
    SUM(area) as area,
    ST_Area(ST_Transform(ST_Union(pdo.mpoly), 2154)) / 10000 as artificial_area
FROM
    public_data_zoneurba pdz
    INNER JOIN project_emprise pe ON ST_Intersects(pe.mpoly, pdz.mpoly) AND pe.project_id = %s
    INNER JOIN public_data_ocsge pdo ON
        ST_Intersects(pdo.mpoly, pdz.mpoly)
        AND pdo.year = '2019'
        AND pdo.is_artificial = true
GROUP BY pdz.typezone
"""
execute(query, [1504])

[('A', None, 2745.148977004338),
 ('AUc', None, 1275.8356004737577),
 ('AUs', None, 856.4687234541727),
 ('Ah', None, 619.8481747461672),
 ('N', None, 2394.978381374791),
 ('Nh', None, 228.72882496663811),
 ('U', None, 3011.836032518168)]